<a href="https://colab.research.google.com/github/entelecheia/ekorpkit-config/blob/main/notebooks/sentiment/financial_phrasebank_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

In [1]:
%%capture
%pip install ekorpkit[model,tokenize,dataset]==0.1.25.post0.dev5

## Prepare `edgar` sampel dataframe

In [2]:
from ekorpkit import eKonf

df_cfg = eKonf.compose(config_group='pipeline=blank')
df_cfg.name = 'edgar_sample'
df_cfg.data_dir = "${cached_path:'gd://15WFOGe73pIDvJEJxV90luirsT4yEInz9:edgar.zip',true,true}"
df_cfg.data_file = 'edgar.parquet'
df_cfg.columns_to_keep = ['id', 'filename', 'item', 'cik', 'company', 'text']
df = eKonf.instantiate(df_cfg)
df.head()

No pipeline specified


,id,filename,item,cik,company,text
1410,1534,1999/320193_10K_1999_0000912057-99-010244.json,item_1,320193,APPLE COMPUTER INC,"ITEM 1. \nBUSINESS GENERAL Apple Computer, Inc..."
1560,1697,1999/21344_10K_1999_0000021344-00-000009.json,item_1,21344,COCA COLA CO,ITEM 1. \nBUSINESS The Coca-Cola Company (toge...
2746,2977,1999/70858_10K_1999_0000950168-00-000621.json,item_1,70858,BANK OF AMERICA CORP /DE/,Item 1. \nBUSINESS General Bank of America Cor...
3762,4088,1999/80424_10K_1999_0000080424-99-000027.json,item_1,80424,PROCTER & GAMBLE CO,Item 1. \nBusiness. \n--------- General Develo...
4806,5211,1999/1018724_10K_1999_0000891020-00-000622.json,item_1,1018724,AMAZON COM INC,ITEM 1. \nBUSINESS This Annual Report on Form ...


## Prepare `financial_phrasebank` dataset

In [2]:
from ekorpkit import eKonf

ds_cfg = eKonf.compose(config_group='dataset=dataset')
ds_cfg.name = 'financial_phrasebank'
ds_cfg.data_dir = "${cached_path:'gd://15TUjIfQstCYXU82s0pcaxh2QcSWf1zyG:financial_phrasebank.zip',true,false}"
ds_cfg.use_name_as_subdir = False
ds = eKonf.instantiate(ds_cfg)

## Compose a config for the LM sentiment analyser class

In [4]:
from ekorpkit import eKonf

config_group='model/sentiment=lm'
model_cfg = eKonf.compose(config_group=config_group)
model_cfg.preprocessor.tokenizer.nltk.lemmatize = True

## Instantiating a sentiment analyser class and prediting sentiments of `edgar` dataset

In [5]:
cfg = eKonf.compose(config_group='pipeline=pipeline')
cfg.verbose = True
cfg.name = 'edgar_sentiments'
cfg.data_dir = "${cached_path:'gd://15WFOGe73pIDvJEJxV90luirsT4yEInz9:edgar.zip',true,true}"
cfg.data_file = 'edgar.parquet'
cfg.columns_to_keep = ['id', 'filename', 'item', 'cik', 'company', 'text']
cfg._pipeline_ = ['predict']
cfg.predict.model = model_cfg
cfg.predict.output_dir = "./data/predict"
cfg.predict.output_file = f'{cfg.name}-lm.parquet'
cfg.num_workers = 100
df = eKonf.instantiate(cfg)
df.head()

Loading 1 dataframes from ['/root/.ekorpkit/.cache/gdown/.15WFOGe73pIDvJEJxV90luirsT4yEInz9/edgar/edgar.parquet']


Predicting [text]: 100%|██████████| 100/100 [00:24<00:00,  4.14it/s]


 >> saved dataframe to ./data/predict/edgar_sentiments-lm.parquet


,id,filename,item,cik,company,text,polarity,subjectivity,polarity_label,uncertainty
1410,1534,1999/320193_10K_1999_0000912057-99-010244.json,item_1,320193,APPLE COMPUTER INC,"ITEM 1. \nBUSINESS GENERAL Apple Computer, Inc...",-0.117647,0.017405,neutral,0.011262
1560,1697,1999/21344_10K_1999_0000021344-00-000009.json,item_1,21344,COCA COLA CO,ITEM 1. \nBUSINESS The Coca-Cola Company (toge...,0.066667,0.008864,neutral,0.014773
2746,2977,1999/70858_10K_1999_0000950168-00-000621.json,item_1,70858,BANK OF AMERICA CORP /DE/,Item 1. \nBUSINESS General Bank of America Cor...,-0.219512,0.011724,negative,0.008579
3762,4088,1999/80424_10K_1999_0000080424-99-000027.json,item_1,80424,PROCTER & GAMBLE CO,Item 1. \nBusiness. \n--------- General Develo...,0.090909,0.008703,neutral,0.011867
4806,5211,1999/1018724_10K_1999_0000891020-00-000622.json,item_1,1018724,AMAZON COM INC,ITEM 1. \nBUSINESS This Annual Report on Form ...,-0.106494,0.031141,neutral,0.019979


In [8]:
print(cfg.predict.output_dir)
print(cfg.predict.output_file)

./data/predict
edgar_sentiments-lm.parquet


## Instantiating a transformer classficiation model with `financial_phrasebank` dataset

In [8]:
from ekorpkit import eKonf

overrides=[
    '+model/transformer=simple_classification',
    '+model/transformer/pretrained=finbert',
]
model_cfg = eKonf.compose(overrides=overrides, config_group='model/transformer=simple_classification')
model_cfg.dataset = ds_cfg
model_cfg.verbose = False
model_cfg.config.num_train_epochs = 2
model_cfg.config.max_seq_length = 256
model_cfg.config.train_batch_size = 32
model_cfg.config.eval_batch_size = 32
model_cfg.labels = ['positive','neutral','negative']
model_cfg._call_ = ['train']
eKonf.instantiate(model_cfg)

  0%|          | 0/1445 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: Currently logged in as: entelecheia (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Running Epoch 0 of 2:   0%|          | 0/46 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/452 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/15 [00:00<?, ?it/s]

Training loss,▁
acc,▁█
eval_loss,█▁
global_step,▁▂█
lr,▁
mcc,▁█
train_loss,█▁
Training loss,0.56368
acc,0.85912
eval_loss,0.34183
global_step,92


wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [9]:
cfg = eKonf.compose(config_group='pipeline=pipeline')
cfg.verbose = True
cfg.name = 'edgar_sentiments'
cfg.data_dir = "${cached_path:'gd://15WFOGe73pIDvJEJxV90luirsT4yEInz9:edgar.zip',true,true}"
cfg.data_file = 'edgar.parquet'
cfg.columns_to_keep = ['id', 'filename', 'item', 'cik', 'company', 'text']
cfg._pipeline_ = ['predict']
cfg.predict.model = model_cfg
cfg.predict.output_dir = "./data/predict"
cfg.predict.output_file = f'{cfg.name}-finbert.parquet'
cfg.num_workers = 1
df = eKonf.instantiate(cfg)
df.head()

Loading 1 dataframes from ['/root/.ekorpkit/.cache/gdown/.15WFOGe73pIDvJEJxV90luirsT4yEInz9/edgar/edgar.parquet']


  0%|          | 0/1445 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Running Epoch 0 of 2:   0%|          | 0/46 [00:00<?, ?it/s]

...

  0%|          | 0/1585 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4252 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/945 [00:00<?, ?it/s]

 >> saved dataframe to ./data/predict/edgar_sentiments-finbert.parquet


,id,filename,item,cik,company,text,pred_labels
1410,1534,1999/320193_10K_1999_0000912057-99-010244.json,item_1,320193,APPLE COMPUTER INC,"ITEM 1. \nBUSINESS GENERAL Apple Computer, Inc...",positive
1560,1697,1999/21344_10K_1999_0000021344-00-000009.json,item_1,21344,COCA COLA CO,ITEM 1. \nBUSINESS The Coca-Cola Company (toge...,neutral
2746,2977,1999/70858_10K_1999_0000950168-00-000621.json,item_1,70858,BANK OF AMERICA CORP /DE/,Item 1. \nBUSINESS General Bank of America Cor...,neutral
3762,4088,1999/80424_10K_1999_0000080424-99-000027.json,item_1,80424,PROCTER & GAMBLE CO,Item 1. \nBusiness. \n--------- General Develo...,neutral
4806,5211,1999/1018724_10K_1999_0000891020-00-000622.json,item_1,1018724,AMAZON COM INC,ITEM 1. \nBUSINESS This Annual Report on Form ...,positive


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1585 entries, 1410 to 1291201
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           1585 non-null   int64 
 1   filename     1585 non-null   object
 2   item         1585 non-null   object
 3   cik          1585 non-null   object
 4   company      1585 non-null   object
 5   text         1585 non-null   object
 6   pred_labels  1585 non-null   object
dtypes: int64(1), object(6)
memory usage: 99.1+ KB
